In [33]:
import numpy as np
import pandas as pd
import glob
import os
import json

from bs4 import BeautifulSoup
from urllib.request import urlopen

import requests

In [21]:
# drug_data.json generated elsewhere
def getDrugInfo():
    ddinter = 1
    url_exists = True
    while url_exists:
        URL = f"http://ddinter.scbdd.com/ddinter/drug-detail/DDInter{ddinter}/"
        if requests.get(URL).status_code == 200:
            # get html
            page = urlopen(URL)
            html = page.read().decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')

            links = soup.find_all('a')

            drug_bank_id = ""
            chembl_id = ""
            pubchem_id = ""

            for link in links:
                link_content = link['href']
                if 'chembl' in link_content:
                    chembl_id = link_content.split(
                        'https://www.ebi.ac.uk/chembl/compound_report_card/')[-1]
                if 'drugbank' in link_content:
                    drug_bank_id = link_content.split(
                        'https://go.drugbank.com/drugs/')[-1]
                if 'pubchem' in link_content:
                    pubchem_id = link_content.split(
                        'https://pubchem.ncbi.nlm.nih.gov/compound/')[-1]

            drug_info = {
                'ddinter': ddinter,
                'drugbank': drug_bank_id,
                'chembl': chembl_id,
                'pubchem': pubchem_id
            }
            ddinter += 1
            yield drug_info
        else:
            url_exists = False
            break


drug_info = getDrugInfo()

data = {'records': []}

for drug in drug_info:
    data['records'].append(drug)
    print(drug)

with open('drug_data.json', 'w') as f:
    json.dump(data, f)

{'ddinter': 1, 'drugbank': 'DB01048', 'chembl': 'CHEMBL1380', 'pubchem': '441300'}
{'ddinter': 2, 'drugbank': 'DB05084', 'chembl': '', 'pubchem': ''}
{'ddinter': 3, 'drugbank': 'DB11932', 'chembl': 'CHEMBL2205807', 'pubchem': '15664'}
{'ddinter': 4, 'drugbank': 'DB00106', 'chembl': 'CHEMBL1252', 'pubchem': '16131215'}
{'ddinter': 5, 'drugbank': 'DB01281', 'chembl': '', 'pubchem': ''}
{'ddinter': 6, 'drugbank': 'DB00054', 'chembl': '', 'pubchem': ''}
{'ddinter': 7, 'drugbank': 'DB12001', 'chembl': 'CHEMBL3301610', 'pubchem': '46220502'}
{'ddinter': 8, 'drugbank': 'DB05812', 'chembl': 'CHEMBL254328', 'pubchem': '132971'}
{'ddinter': 9, 'drugbank': 'DB11703', 'chembl': 'CHEMBL3707348', 'pubchem': '71226662'}
{'ddinter': 10, 'drugbank': 'DB00659', 'chembl': 'CHEMBL1201293', 'pubchem': '71158'}
{'ddinter': 11, 'drugbank': 'DB00284', 'chembl': '', 'pubchem': '9811704'}
{'ddinter': 12, 'drugbank': 'DB01193', 'chembl': 'CHEMBL642', 'pubchem': '1978'}
{'ddinter': 13, 'drugbank': 'DB01418', 'che

ConnectionError: HTTPConnectionPool(host='ddinter.scbdd.com', port=80): Max retries exceeded with url: /ddinter/drug-detail/DDInter25/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000232A33053D0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [34]:
def load_data(data_folder):
    csv_files = os.path.join(data_folder, "ddinter_downloads_code_*.csv")
    joined_csv_files = glob.glob(csv_files)
    merged_csv = pd.concat(
        map(pd.read_csv, joined_csv_files), ignore_index=True)

    # load file with scraped data
    drug_info_file = os.path.join(data_folder, 'drug_data.json')
    drug_info = open(drug_info_file)
    drug_data = json.load(drug_info)['records']

    drug_characteristics = ['chembl', 'pubchem', 'drugbank']

    ids = []

    for index in merged_csv.index:
        DDInterID_A = merged_csv['DDInterID_A'][index]
        Drug_A = merged_csv['Drug_A'][index]
        DDInterID_A_index = int(DDInterID_A.split('DDInter')[-1])
        DDInterID_B = merged_csv['DDInterID_B'][index]
        Drug_B = merged_csv['Drug_B'][index]
        DDInterID_B_index = int(DDInterID_B.split('DDInter')[-1])
        Level = merged_csv['Level'][index]
        id = DDInterID_A+'_'+DDInterID_B+'_'+Level

        if id not in ids:
            doc = {}
            doc['_id'] = id
            doc['drug_a'] = {
                'ddinterid': DDInterID_A,
                'name': Drug_A,
            }

            for characteristic in drug_characteristics:
                info = drug_data[DDInterID_A_index-1][characteristic]
                if len(info) > 0:
                    doc['drug_a'][characteristic] = info

            doc['drug_b'] = {
                'ddinterid': DDInterID_B,
                'name': Drug_B,
            }

            for characteristic in drug_characteristics:
                info = drug_data[DDInterID_B_index-1][characteristic]
                if len(info) > 0:
                    doc['drug_b'][characteristic] = info

            doc['level'] = Level

            ids.append(id)

            yield doc
        else:
            continue

In [99]:
data = load_data('ddinter_data')

In [91]:
df = pd.DataFrame()
curr_messy_dict = data.__next__()
drug_a_chembl = ''
drug_a_pubchem = ''
drug_a_drugbank = ''

if curr_messy_dict['drug_a'].get('chembl') != None:
    drug_a_chembl = curr_messy_dict['drug_a']['chembl']
else:
    drug_a_chembl = ''
    
if curr_messy_dict['drug_a'].get('pubchem') != None:
    drug_a_pubchem = curr_messy_dict['drug_a']['pubchem']
else:
    drug_a_pubchem = ''

if curr_messy_dict['drug_a'].get('drugbank') != None:
    drug_a_drugbank = curr_messy_dict['drug_a']['drugbank']
else:
    drug_a_drugbank = ''
    
drug_b_chembl = ''
drug_b_pubchem = ''
drug_b_drugbank = ''

if curr_messy_dict['drug_b'].get('chembl') != None:
    drug_b_chembl = curr_messy_dict['drug_b']['chembl']
else:
    drug_a_chembl = ''
    
if curr_messy_dict['drug_b'].get('pubchem') != None:
    drug_b_pubchem = curr_messy_dict['drug_b']['pubchem']
else:
    drug_b_pubchem = ''

if curr_messy_dict['drug_b'].get('drugbank') != None:
    drug_b_drugbank = curr_messy_dict['drug_b']['drugbank']
else:
    drug_a_drugbank = ''

curr_clean_dict = {'drug_a': [curr_messy_dict['drug_a']['name']], 
                   'drug_a_chembl': [drug_a_chembl],
                   'drug_a_pubchem': [drug_a_pubchem],
                   'drug_a_drugbank' : [drug_a_drugbank],
                   'drug_b': [curr_messy_dict['drug_b']['name']], 
                   'drug_b_chembl': [drug_b_chembl],
                   'drug_b_pubchem': [drug_b_pubchem],
                   'drug_b_drugbank' : [drug_b_drugbank],
                   'interaction_level': [curr_messy_dict['level']]}
df1 = pd.DataFrame.from_dict(curr_clean_dict)

df = df.append(df1, ignore_index=True)

curr_messy_dict = data.__next__()
drug_a_chembl = ''
drug_a_pubchem = ''
drug_a_drugbank = ''

if curr_messy_dict['drug_a'].get('chembl') != None:
    drug_a_chembl = curr_messy_dict['drug_a']['chembl']
else:
    drug_a_chembl = ''
    
if curr_messy_dict['drug_a'].get('pubchem') != None:
    drug_a_pubchem = curr_messy_dict['drug_a']['pubchem']
else:
    drug_a_pubchem = ''

if curr_messy_dict['drug_a'].get('drugbank') != None:
    drug_a_drugbank = curr_messy_dict['drug_a']['drugbank']
else:
    drug_a_drugbank = ''
    
drug_b_chembl = ''
drug_b_pubchem = ''
drug_b_drugbank = ''

if curr_messy_dict['drug_b'].get('chembl') != None:
    drug_b_chembl = curr_messy_dict['drug_b']['chembl']
else:
    drug_a_chembl = ''
    
if curr_messy_dict['drug_b'].get('pubchem') != None:
    drug_b_pubchem = curr_messy_dict['drug_b']['pubchem']
else:
    drug_b_pubchem = ''

if curr_messy_dict['drug_b'].get('drugbank') != None:
    drug_b_drugbank = curr_messy_dict['drug_b']['drugbank']
else:
    drug_a_drugbank = ''

curr_clean_dict = {'drug_a': [curr_messy_dict['drug_a']['name']], 
                   'drug_a_chembl': [drug_a_chembl],
                   'drug_a_pubchem': [drug_a_pubchem],
                   'drug_a_drugbank' : [drug_a_drugbank],
                   'drug_b': [curr_messy_dict['drug_b']['name']], 
                   'drug_b_chembl': [drug_b_chembl],
                   'drug_b_pubchem': [drug_b_pubchem],
                   'drug_b_drugbank' : [drug_b_drugbank],
                   'interaction_level': [curr_messy_dict['level']]}

df2 = pd.DataFrame.from_dict(curr_clean_dict)

df = df.append(df2, ignore_index=True)
df

C:\Users\dell\AppData\Local\Temp\ipykernel_5288\2529610485.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1, ignore_index=True)
C:\Users\dell\AppData\Local\Temp\ipykernel_5288\2529610485.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


drug_a  drug_a_chembl drug_a_pubchem drug_a_drugbank  \
0  Dolutegravir  CHEMBL1229211       54726191         DB08930   
1  Dolutegravir  CHEMBL1229211       54726191         DB08930   

                         drug_b  drug_b_chembl drug_b_pubchem drug_b_drugbank  \
0  Calcium glubionate anhydrous  CHEMBL2219740      131704325         DB13142   
1             Calcium gluconate  CHEMBL2106119           9290         DB11126   

  interaction_level  
0             Major  
1             Major

In [100]:
df = pd.DataFrame()
for interaction in data:
    drug_a_chembl = ''
    drug_a_pubchem = ''
    drug_a_drugbank = ''
    
    curr_messy_dict = interaction

    if curr_messy_dict['drug_a'].get('chembl') != None:
        drug_a_chembl = curr_messy_dict['drug_a']['chembl']
    else:
        drug_a_chembl = ''

    if curr_messy_dict['drug_a'].get('pubchem') != None:
        drug_a_pubchem = curr_messy_dict['drug_a']['pubchem']
    else:
        drug_a_pubchem = ''

    if curr_messy_dict['drug_a'].get('drugbank') != None:
        drug_a_drugbank = curr_messy_dict['drug_a']['drugbank']
    else:
        drug_a_drugbank = ''

    drug_b_chembl = ''
    drug_b_pubchem = ''
    drug_b_drugbank = ''

    if curr_messy_dict['drug_b'].get('chembl') != None:
        drug_b_chembl = curr_messy_dict['drug_b']['chembl']
    else:
        drug_b_chembl = ''

    if curr_messy_dict['drug_b'].get('pubchem') != None:
        drug_b_pubchem = curr_messy_dict['drug_b']['pubchem']
    else:
        drug_b_pubchem = ''

    if curr_messy_dict['drug_b'].get('drugbank') != None:
        drug_b_drugbank = curr_messy_dict['drug_b']['drugbank']
    else:
        drug_b_drugbank = ''

    curr_clean_dict = {'drug_a': [curr_messy_dict['drug_a']['name']], 
                       'drug_a_chembl': [drug_a_chembl],
                       'drug_a_pubchem': [drug_a_pubchem],
                       'drug_a_drugbank' : [drug_a_drugbank],
                       'drug_b': [curr_messy_dict['drug_b']['name']], 
                       'drug_b_chembl': [drug_b_chembl],
                       'drug_b_pubchem': [drug_b_pubchem],
                       'drug_b_drugbank' : [drug_b_drugbank],
                       'interaction_level': [curr_messy_dict['level']]}
    curr_df = pd.DataFrame.from_dict(curr_clean_dict)
    df = df.append(curr_df, ignore_index=True)
df

C:\Users\dell\AppData\Local\Temp\ipykernel_5288\3543873738.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(curr_df, ignore_index=True)


drug_a  drug_a_chembl drug_a_pubchem drug_a_drugbank  \
0               Naltrexone    CHEMBL19019        5360515         DB00704   
1                 Abacavir     CHEMBL1380         441300         DB01048   
2       Aluminum hydroxide  CHEMBL1200706       10176082         DB06723   
3               Aprepitant     CHEMBL1471        6918365         DB00673   
4              Attapulgite                      73415752         DB01574   
...                    ...            ...            ...             ...   
160230          Carvedilol      CHEMBL723           2585         DB01136   
160231           Ofloxacin        CHEMBL4           4583         DB01165   
160232         Ceftriaxone      CHEMBL161        5479530         DB01212   
160233          Quetiapine      CHEMBL716           5002         DB01224   
160234            Cefepime      CHEMBL186        5479537         DB01413   

              drug_b  drug_b_chembl drug_b_pubchem drug_b_drugbank  \
0           Abacavir     CHEMBL1380         441300         DB01048   
1           Orlistat   CHEMBL175247        3034010         DB01083   
2       Dolutegravir  CHEMBL1229211       54726191         DB08930   
3       Dolutegravir  CHEMBL1229211       54726191         DB08930   
4       Dolutegravir  CHEMBL1229211       54726191         DB08930   
...              ...            ...            ...             ...   
160230     Galactose                       3037556         DB11735   
160231     Galactose                       3037556         DB11735   
160232     Galactose                       3037556         DB11735   
160233     Galactose                       3037556         DB11735   
160234     Galactose                       3037556         DB11735   

       interaction_level  
0               Moderate  
1               Moderate  
2                  Major  
3                  Minor  
4                  Major  
...                  ...  
160230           Unknown  
160231           Unknown  
160232           Unknown  
160233           Unknown  
160234           Unknown  

[160235 rows x 9 columns]

In [101]:
df.to_csv('all_ddi_w_ids.csv', index=False)

In [54]:
print(interaction['drug_a']['name'])
print(interaction['drug_a']['chembl'])
print(interaction['drug_a']['pubchem'])
print(interaction['drug_a']['drugbank'])

Abacavir
CHEMBL1380
441300
DB01048


In [96]:
for dat in data:
    print(dat)